# **Notebook From https://github.com/TheLastBen/fast-stable-diffusion, if you encounter any issues, feel free to discuss them.** [Support](https://ko-fi.com/thelastben)

In [ ]:
#only for freash install
!rm -r /kaggle/working/MyDrive

In [ ]:
#@markdown # Connect To Kaggle Drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
print("[0;33mConnecting...")
mainpth="MyDrive"
Shared_Drive=""
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Install/Update camenduru/AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64

Use_Latest_Working_Commit= False #@param {type:"boolean"}

#@markdown - Use the pre-major A1111 update commit to avoid bugs while the repo is fixed

blasphemy=base64.b64decode(("d2VidWk=").encode('ascii')).decode('ascii')

if not os.path.exists("/kaggle/working"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /kaggle/working/$mainpth
  Shared_Drive=""
  
if Shared_Drive!="" and not os.path.exists("/kaggle/working/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /kaggle/working/$mainpth/sd
  %cd /kaggle/working/$mainpth/sd
  !git clone -q --branch master https://github.com/camenduru/stable-diffusion-$blasphemy
  !mkdir -p /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/cache/
  os.environ['TRANSFORMERS_CACHE']=f"/kaggle/working/{mainpth}/sd/stable-diffusion-"+blasphemy+"/cache"

with capture.capture_output() as cap:
  %cd /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
  !git pull
  if Use_Latest_Working_Commit:
    !git fetch --unshallow
    !git checkout a9eab236d7e8afa4d6205127904a385b2c43bb24
    !pip install gradio==3.16.2 --no-deps -qq
  else:
    !pip install gradio==3.23 --no-deps -qq
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /kaggle/working/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stablediffusion'):
    !tar -C /kaggle/working/$mainpth --zstd -xf sd_rep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  !wget -q -O /usr/local/lib/python3.9/dist-packages/flax/core/frozen_dict.py  https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/frozen_dict.py
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  if Use_Latest_Working_Commit:
    !pip install gradio==3.16.2 --no-deps -qq
  else:
    !pip install gradio==3.23 --no-deps -qq

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Model Download/Load
Model_Version = "V2.1-768px" #@param [ "1.5", "v1.5 Inpainting ", "V2.1-512px", "V2.1-768px"]

Redownload_the_original_model = False #@param {type:"boolean"}

def rmv():
  !wget -q -O /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/models/Stable-diffusion/model.ckpt https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/README.md
  !mv /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/models/Stable-diffusion/model.ckpt /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/models/Stable-diffusion/trashfile.f
  time.sleep(2)
  !rm /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/models/Stable-diffusion/trashfile.f
  clear_output()

if Redownload_the_original_model:
  with capture.capture_output() as cap:
    rmv()

#@markdown Or
Path_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full path of your trained model or to a folder containing multiple models

#@markdown Or
MODEL_LINK = "" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}
Use_temp_storage = False #@param {type:"boolean"}



def newmdl():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-v1-5"
      if os.path.exists('/kaggle/working/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
        !wget -q -O stable-diffusion-v1-5/vae/diffusion_pytorch_model.bin https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin
        !wget -q -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/kaggle/working/stable-diffusion-v1-5"@' /kaggle/working/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/kaggle/working/{mainpth}/sd/stable-diffusion-{blasphemy}/models/Stable-diffusion/model.ckpt"@' /kaggle/working/convertosd.py
        clear_output()       
        !python /kaggle/working/convertosd.py
        !rm /kaggle/working/convertosd.py
        if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'):
          model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")

    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V1.5','primary', '700px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git

    if os.path.exists('/kaggle/working/stable-diffusion-v1-5'):
      !rm -r /kaggle/working/stable-diffusion-v1-5

    return model

def V2():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !mkdir "/kaggle/working/stable-diffusion-V2"
      %cd "/kaggle/working/stable-diffusion-V2"
      !git init
      !git lfs install --system --skip-repo
      if Model_Version == "V2.1-768px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
      elif Model_Version == "V2.1-512px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
      !git config core.sparsecheckout true
      !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
      !git pull origin fp16
      %cd /kaggle/working
      !wget -O convertosdv2.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosdv2.py
      clear_output()
      !python /kaggle/working/convertosdv2.py --fp16 /kaggle/working/stable-diffusion-V2 /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/models/Stable-diffusion/model.ckpt
      !rm /kaggle/working/convertosdv2.py
      if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'):
        model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'
        clear_output()
        inf('\u2714 Done','success', '50px')
      else:
        inf('\u2718 Something went wrong, try again','danger', "250px")

    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V2','primary', '700px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git
      !rm -r /kaggle/working/convertosdv2.py
    if os.path.exists('/kaggle/working/stable-diffusion-V2'):
      !rm -r /kaggle/working/stable-diffusion-V2

    return model

def inpmdl():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-inpainting"
      if os.path.exists('/kaggle/working/stable-diffusion-inpainting'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /kaggle/working/stable-diffusion-inpainting/vae
        !mv /kaggle/working/sd-vae-ft-mse /kaggle/working/stable-diffusion-inpainting/vae        
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/kaggle/working/stable-diffusion-inpainting"@' /kaggle/working/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/kaggle/working/{mainpth}/sd/stable-diffusion-{blasphemy}/models/Stable-diffusion/sd-v1-5-inpainting.ckpt"@' /kaggle/working/convertosd.py
        clear_output()       
        !python /kaggle/working/convertosd.py
        !rm /kaggle/working/convertosd.py
        if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
          model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")


    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
      clear_output()
      inf('\u2714 inpainting model already exists','primary', '250px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git

    if os.path.exists('/kaggle/working/stable-diffusion-inpainting'):
      !rm -r /kaggle/working/stable-diffusion-inpainting

    return model

if (Path_to_MODEL !=''):
  if os.path.exists(str(Path_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(Path_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        Path_to_MODEL=input()
      if os.path.exists(str(Path_to_MODEL)):
        inf('\u2714 Using the trained model.','success', '200px')

  model=Path_to_MODEL

elif MODEL_LINK != "":
  gdrv='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/models/Stable-diffusion'
  tmp="/kaggle/working"
  pth=tmp if Use_temp_storage else gdrv
  %cd $pth
  clear_output()
  if not safetensors:
    modelname="model.ckpt"
  else:
    modelname="model.safetensors"
  !gdown --fuzzy -O $modelname "$MODEL_LINK"
  if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
      model=f'{pth}/{modelname}'
      clear_output()
      inf('\u2714 Model downloaded, using the trained model.','success', '350px')
  else:
    if Use_temp_storage:
      !rm $pth/$modelname
    else:
      rmv()
    inf('\u2718 Wrong link, check that the link is valid','danger', "300px")


elif Model_Version=="1.5":
  model=newmdl()

elif Model_Version=="V2.1-512px" or Model_Version=="V2.1-768px":
  model=V2()

else:
   model=inpmdl()

In [ ]:
!pip install gdown
#@markdown # LORA/Model Download/Load

islora = False #@param {type:"boolean"}
Path_to_MODEL = "abyssorangemix3AOM3_aom3" #@param {type:"string"}
#@markdown - Insert the full NAME of LORA model

MODEL_LINK = "https://huggingface.co/hanafuusen2001/AbyssOrangeMix_AOM/resolve/main/abyssorangemix3AOM3_aom3.safetensors"
safetensors = True #@param {type:"boolean"}
Use_temp_storage = False #@param {type:"boolean"}



def newmdl():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-v1-5"
      if os.path.exists('/kaggle/working/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
        !wget -q -O stable-diffusion-v1-5/vae/diffusion_pytorch_model.bin https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin
        !wget -q -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/kaggle/working/stable-diffusion-v1-5"@' /kaggle/working/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/kaggle/working/{mainpth}/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt"@' /kaggle/working/convertosd.py
        clear_output()       
        !python /kaggle/working/convertosd.py
        !rm /kaggle/working/convertosd.py
        if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
          model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")

    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V1.5','primary', '700px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git

    if os.path.exists('/kaggle/working/stable-diffusion-v1-5'):
      !rm -r /kaggle/working/stable-diffusion-v1-5

    return model

def V2():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !mkdir "/kaggle/working/stable-diffusion-V2"
      %cd "/kaggle/working/stable-diffusion-V2"
      !git init
      !git lfs install --system --skip-repo
      if Model_Version == "V2.1-768px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
      elif Model_Version == "V2.1-512px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
      !git config core.sparsecheckout true
      !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
      !git pull origin fp16
      %cd /kaggle/working
      !wget -O convertosdv2.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosdv2.py
      clear_output()
      !python /kaggle/working/convertosdv2.py --fp16 /kaggle/working/stable-diffusion-V2 /kaggle/working/$mainpth/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
      !rm /kaggle/working/convertosdv2.py
      if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
        model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
        clear_output()
        inf('\u2714 Done','success', '50px')
      else:
        inf('\u2718 Something went wrong, try again','danger', "250px")

    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V2','primary', '700px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git
      !rm -r /kaggle/working/convertosdv2.py
    if os.path.exists('/kaggle/working/stable-diffusion-V2'):
      !rm -r /kaggle/working/stable-diffusion-V2

    return model

def inpmdl():

    if not os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
      %cd /kaggle/working/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-inpainting"
      if os.path.exists('/kaggle/working/stable-diffusion-inpainting'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /kaggle/working/stable-diffusion-inpainting/vae
        !mv /kaggle/working/sd-vae-ft-mse /kaggle/working/stable-diffusion-inpainting/vae        
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/kaggle/working/stable-diffusion-inpainting"@' /kaggle/working/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/kaggle/working/{mainpth}/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt"@' /kaggle/working/convertosd.py
        clear_output()       
        !python /kaggle/working/convertosd.py
        !rm /kaggle/working/convertosd.py
        if os.path.exists('/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
          model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")


    else:
      model='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
      clear_output()
      inf('\u2714 inpainting model already exists','primary', '250px')

    if os.path.exists('/kaggle/working/.git'):
      !rm -r /kaggle/working/.git

    if os.path.exists('/kaggle/working/stable-diffusion-inpainting'):
      !rm -r /kaggle/working/stable-diffusion-inpainting

    return model
# /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion
# /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Lora
if (MODEL_LINK != ""):
    gdrv='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Stable-diffusion'

    if(islora):
        gdrv='/kaggle/working/'+mainpth+'/sd/stable-diffusion-webui/models/Lora'

    tmp="/kaggle/working"
    pth=tmp if Use_temp_storage else gdrv
    %cd $pth
    clear_output()

    if (Path_to_MODEL !=''):
      modelnvvv = str(Path_to_MODEL)
    else:
      modelnvvv= 'model1'  
    if not safetensors:
      modelname=str(modelnvvv)+".ckpt"
    else:
      modelname=str(modelnvvv)+".safetensors"


    !gdown --fuzzy -O $modelname "$MODEL_LINK"
    if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
        model=f'{pth}/{modelname}'
        clear_output()
        inf('\u2714 Model downloaded, using the trained model.','success', '350px')
    else:
      if Use_temp_storage:
        !rm $pth/$modelname
      else:
        rmv()
      inf('\u2718 Wrong link, check that the link is valid','danger', "300px")

In [ ]:
#@markdown # ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse
import re
from subprocess import run

Model = "OpenPose" #@param [ "None", "All (21GB)", "Canny", "Depth", "Lineart", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "ip2p", "Shuffle", "Inpaint", "Softedge", "Lineart_Anime", "Tile", "T2iadapter_Models"]

v2_Model = "OpenPose" #@param [ "None", "All", "Canny", "Depth", "HED", "OpenPose", "Scribble"]

#@markdown - Download/update ControlNet extension and its models

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth'
Depth='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth'
Lineart='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth'
MLSD='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth'
Normal='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth'
OpenPose='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth'
Scribble='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth'
Seg='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth'
ip2p='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth'
Shuffle='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth'
Inpaint='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth'
Softedge='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth'
Lineart_Anime='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth'
Tile='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11u_sd15_tile.pth'

with capture.capture_output() as cap:
  %cd /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-$blasphemy-controlnet.git
    %cd /kaggle/working
  else:
    %cd sd-webui-controlnet
    !git reset --hard
    !git pull
    %cd /kaggle/working

mdldir='/kaggle/working/'+mainpth+'/sd/stable-diffusion-'+blasphemy+'/extensions/sd-webui-controlnet/models'
for filename in os.listdir(mdldir):
  if "_sd14v1" in filename:
    renamed = re.sub("_sd14v1", "-fp16", filename)
    os.rename(os.path.join(mdldir, filename), os.path.join(mdldir, renamed))

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt
!wget -q -O CN_models_v2.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models_v2.txt

with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()
with open("CN_models_v2.txt", 'r') as d:
  mdllnk_v2 = d.read().splitlines()

!rm CN_models.txt CN_models_v2.txt

with capture.capture_output() as cap:
  cfgnames=[os.path.basename(url).split('.')[0]+'.yaml' for url in mdllnk_v2]
  %cd /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/extensions/sd-webui-controlnet/models
  for name in cfgnames:
      run(['cp', 'cldm_v21.yaml', name])
  %cd /kaggle/working

if Model == "All (21GB)": 
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()

elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()

elif Model == "None":
    pass
    clear_output()

else:
  download(globals()[Model], mdldir)
  clear_output()

Canny='https://huggingface.co/thibaud/controlnet-sd21/resolve/main/control_v11p_sd21_canny.safetensors'
Depth='https://huggingface.co/thibaud/controlnet-sd21/resolve/main/control_v11p_sd21_depth.safetensors'
HED='https://huggingface.co/thibaud/controlnet-sd21/resolve/main/control_v11p_sd21_hed.safetensors'
OpenPose='https://huggingface.co/thibaud/controlnet-sd21/resolve/main/control_v11p_sd21_openpose.safetensors'
Scribble='https://huggingface.co/thibaud/controlnet-sd21/resolve/main/control_v11p_sd21_scribble.safetensors'

if v2_Model == "All":
  for lnk_v2 in mdllnk_v2:
      download(lnk_v2, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

elif v2_Model == "None":
    pass
    clear_output()
    inf('\u2714 Done','success', '50px')

else:
  download(globals()[v2_Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Uninstall New Libs And Re-install Old Ones (NANS FIX)
!pip uninstall xformers -y
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip uninstall torchaudio -y
!pip uninstall torchtext -y
!pip uninstall torchdata -y


!pip install torch torchvision torchaudio torchdata torchtext --extra-index-url https://download.pytorch.org/whl/cu117
!pip install xformers==0.0.17

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
!nvidia-smi

In [ ]:
#@markdown # DOWNNLOAD extensions and embeddings from Drive

!rm -r /kaggle/working/MyDrive/sd/stable-diffusion-webui/embeddings
!gdown https://drive.google.com/drive/folders/1ESNCaCUKQY4Sigvd-sLGLjArWmhoZDrU -O /kaggle/working/MyDrive/sd/stable-diffusion-webui --folder
    
clear_output()
inf('\u2714 Done','success', '50px')    

In [ ]:
!ls /kaggle/working/MyDrive/sd/stable-diffusion-webui/embeddings

import torch
print(torch.cuda.device_count())

In [ ]:
!apt -y update -qq
!apt -y install -qq aria2
!pip install -q torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl

In [ ]:
#@markdown # Start Stable-Diffusion
!apt -y update -qq
!apt -y install -qq aria2
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /kaggle/working/MyDrive/sd/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/Mikubill/sd-webui-controlnet /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/camenduru/openpose-editor /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/openpose-editor    
    
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/multidiffusion-upscaler-for-automatic1111
    
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /kaggle/working/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-additional-networks
%cd /kaggle/working/MyDrive/sd/stable-diffusion-webui
!pip install gradio==3.23 --no-deps -qq

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v4.5-vae-swapped/resolve/main/anything-v4.5-vae-swapped.safetensors -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-vae-swapped.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.0.vae.pt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion -o orangemix.vae.pt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ElChuaso/ReV_Animated/resolve/main/revAnimated_v122.safetensors -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion -o revAnimated_v122.safetensors
    
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hanafuusen2001/AbyssOrangeMix_AOM/resolve/main/abyssorangemix3AOM3_aom3.safetensors -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion -o abyssorangemix3AOM3_aom3.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth -d /kaggle/working/MyDrive/sd/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
    

    
!sed -i -e 's/numpy==1.23.3/numpy==1.21.6/g' requirements_versions.txt
!sed -i -e 's/blendmodes==2022/blendmodes==2021/g' requirements_versions.txt
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' requirements_versions.txt
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /kaggle/working/MyDrive/sd/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /kaggle/working/MyDrive/sd/stable-diffusion-webui/launch.py

!npm install -g localtunnel
import subprocess
import threading
import time
import socket
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

# !export CUDA_VISIBLE_DEVICES=0,1

#origanl one
# !python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue

#mine
!python /kaggle/working/$mainpth/sd/stable-diffusion-$blasphemy/launch.py --xformers --no-half --enable-insecure-extension-access --opt-split-attention --share --api --disable-safe-unpickle --no-download-sd-model --theme dark --gradio-queue
    

Applying xformers cross attention optimization.
No Image data blocks found.
Textual inversion embeddings loaded(19): EasyNegative, bad-hands-5, bad-image-v2, ulzzang-6500-v1.1, nixeu_extra, nixeu_basic, nixeu_basic2, nixeu_white, bad-image-v2-39000, bad-artist-anime, badhandv4, bad-artist, verybadimagenegative_v1.3, nixeu_soft, bad-picture-chill-75v, bad_prompt_version2, pureerosface_v1, ng_deepnegative_v1_75t, bad-prompt-v2
Model loaded in 44.1s (load weights from disk: 0.6s, create model: 11.4s, apply weights to model: 27.1s, load VAE: 4.5s, load textual inversion embeddings: 0.5s).
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9b9f85b8a2c49fac88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
Startup time: 69.6s (import torch: 10.8s, import gradio: 1.4s, import ldm: 0.7s, other imports: 1.5s, setup codeformer: 0.1s, load scripts: 2.3s, load SD checkpoin